# ODI Queensland workshop - Web Scraping 

## QUT DMRC - 2015

### Extract all road sign data from a single page

Extend from getting just the sign name to getting all the fields we are interested in from each item.

In [ ]:
import bs4
import requests

In [ ]:
# this is the base_url
base_url = "http://www.qld.gov.au/transport/safety/signs/"

In [ ]:
# select which page to scrape based on the type of road sign
sign_type = "regulatory"

In [ ]:
# build the url
thepage = base_url + sign_type + '/'

In [ ]:
# call the url
stuff = requests.get(thepage)

In [ ]:
# transform to soup using lxml parser
soup = bs4.BeautifulSoup(stuff.text, "lxml")

Now process ```lotsofitems``` in a new way to get more fields for all of the items.

In [ ]:
# in this step we'll scrape sign image url, sign description in addition to sign name

# find all the tables on the page
tables = soup.findAll('table')

# move outside the loop so we just make one list
thelist = []

for table in tables:
    # find all the table rows
    lotsofitems = table.findAll('tr')
    
    # check if the first row contains a 'th' elements (table header)
    if lotsofitems[0].find('th'): 

        # get all header elements
        temp = lotsofitems[0].findAll('th')
        
        # check that the table header has the text we expect for the signs table
        if temp[0].get_text() == 'Sign' and temp[1].get_text() == 'Meaning':
            
            print('Traffic sign table found')
            
            # process the table of traffic signs **** THIS IS THE UPDATED SECTION *****
            for an_item in lotsofitems[1:]: 
                theitem = []

                # sign description & title
                sign_text = an_item.findAll("p")
                description = ''
                for para in sign_text:
                    if para.find("strong"):
                        # extract sign name (this assumes only the sign name is in bold)
                        temptemp = para.find("strong").get_text()
                        temptemp = temptemp.split()
                        sign_name = " ".join(temptemp)
                    else:
                        # extract sign description (may be multiple paragraphs)
                        temptemp = para.get_text()
                        temptemp = temptemp.split()
                        description += " ".join(temptemp) + '\n'

                theitem += [sign_name]
                theitem += [description]

                # sign images (may be more than one image per sign name) - save image name & image url
                images = []
                for image in  an_item.findAll("img"):
                    # get the image name & image url
                    images += [[image.attrs['alt'], image.attrs['src']]]
                theitem += [images]
    
                thelist += [theitem]
    
        else:
            print('Different table - with header row:', temp)
    else:
        print('Different table - no header row:', lotsofitems[0])


In [ ]:
thelist[3]

Now we are ready to move onto the fourth notebook - [Structure the data extraction as a function](step4.ipynb)